In [213]:
# Import modules.
import os
import matplotlib.pyplot as mp

# Import tools for keras.
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

from scipy import stats

import datetime
import time

import copy

In [214]:
from imp import reload
import TensorFlowUtils
reload(TensorFlowUtils)
import SeisUtils
reload(SeisUtils)
import ProgressiveGAN
reload(ProgressiveGAN)
import GAN
reload(GAN)
import ModelBase
reload(ModelBase)
import GANModel
reload(GANModel)
import ParserUtils
reload(ParserUtils)

<module 'ParserUtils' from 'C:\\Users\\micha\\Documents\\CS101\\seis-GAN\\ParserUtils.py'>

In [215]:
from TensorFlowUtils import show_graph
from SeisUtils       import extract_func
from SeisUtils       import SeisData
from SeisUtils       import SeisGenerator
from GAN             import GAN
from ModelBase       import ModelBase
from GANModel        import GANModel
from ParserUtils     import ArgModelParser
from ProgressiveGAN  import ProgressiveGAN

In [216]:
config_d = {
    
    #---------
    # Data
    #---------
    
    'data_path'   : 'wformMat_jpm4p_181106_downsample-5x.h5', # path to data
    'data_format' : 'channels_last', # data format ('channels_first' or 'channels_last')
    'train_frac'  : 0.0001,  # % of data devoted to Training
    
    #---------
    # Wforms
    #---------

    'burn_seconds'                 : 2.5,   # first  part of wform to throw away
    'input_seconds'                : 80,    # middle part of waveform to use as input
    'output_seconds'               : None,  # last   part of waveform to use as target output or None if generting x.
    'measure_rate'                 : 20,    # sampling rate in HZ
    'normalize_data'               : True,  # whether to normalize input waveforms to lie between -1 and 1.
    'predict_normalization_factor' : True, # whether to predict the normalization factor if computed

    #---------
    # Training
    #---------
    
    'batch_size'  : 4, # batch size
    'epochs'      : 2, # training epochs
    'random_seed' : 7, # random seed
    'metas'       : ['dist', 'magn'], # meta to include as conditional parameters.
    'weight_loss' : False, 
    "conditional_config"    : {"one_hot_encode": True, "aux_classify": False},
    "bins_s"                : [10, [0, 3, 4, 5, 6, 7, 8, "inf"]],
    
    #--------
    # Saving
    #--------
    
    'directory'   : 'TEST', # defaults to todays date. Will make a dir called 'GAN_<directory>
    'restore'     : False,
    
    #--------
    # Debug
    #--------

    'debug' : True,

}

In [217]:
gm = GANModel(config_d)

config {'data_path': 'wformMat_jpm4p_181106_downsample-5x.h5', 'data_format': 'channels_last', 'train_frac': 0.0001, 'burn_seconds': 2.5, 'input_seconds': 80, 'output_seconds': None, 'measure_rate': 20, 'normalize_data': True, 'predict_normalization_factor': True, 'batch_size': 4, 'epochs': 2, 'random_seed': 7, 'metas': ['dist', 'magn'], 'weight_loss': False, 'conditional_config': {'one_hot_encode': True, 'aux_classify': False}, 'bins_s': [10, [0, 3, 4, 5, 6, 7, 8, 'inf']], 'directory': 'TEST', 'restore': False, 'debug': True}
Testing    Samples: 144855
Validation Samples: 115884
Training   Samples: 25 

conditional_metav (shape): [(260764, 10), (260764, 7)]
weights: [0.7495804 1.4531128 1.4531128 ... 2.3466036 2.3466036 2.3466036]
   max  =  5014.6924
   min  =  0.084483586
   mean =  0.9999999
   std  =  12.046692 

Testing    Samples: 36214
Validation Samples: 28971
Training   Samples: 6
Generator.call() training = Tensor("training_bool:0", shape=(), dtype=bool, device=/device:GPU:0

In [11]:
# %%pixie_debugger
gm.train()

Getting   randomly sampled batch
Retreived randomly sampled batch
Getting   randomly sampled batch
Retreived randomly sampled batch
Getting   randomly sampled batch
Retreived randomly sampled batch
Getting   randomly sampled batch
Retreived randomly sampled batch
Getting   randomly sampled batch
Retreived randomly sampled batch
Getting   randomly sampled metav.
Retreived randomly sampled metav.
Getting   randomly sampled batch
Retreived randomly sampled batch
     ... Saving summaries at global step: 1
Getting   randomly sampled batch
Retreived randomly sampled batch
     ... Saving summaries at global step: 1
Getting   randomly sampled batch
Retreived randomly sampled batch
     ... Saving summaries at global step: 1
Getting   randomly sampled batch
Retreived randomly sampled batch
     ... Saving summaries at global step: 1
Getting   randomly sampled batch
Retreived randomly sampled batch
     ... Saving summaries at global step: 1
Getting   randomly sampled metav.
Retreived randomly

### Show tf graph 

In [218]:
# show_graph(gm.graph.as_graph_def())

### Attempt to run specific training ops

In [220]:
# Get batch of data.
batch = gm.SG_train.random_batch()
wform_x = batch.x
batch.x.shape, gm.noise().shape

Getting   randomly sampled batch
Retreived randomly sampled batch


((4, 1600, 3), (4, 100, 3))

In [223]:
# Get global step.
with gm.sess.as_default():
    with gm.graph.as_default():
        global_step = tf.train.get_or_create_global_step()
gs_assign   = global_step.assign(13000)

In [224]:
def feed_dict(training = True):
    feed_dict = {gm.gen_input_placeholder : gm.noise(), "real_input:0" : wform_x, gm.training_bool_placeholder : training} #gm.dis_input_placeholder : wform_x}
    if gm.conditional_placeholders is not None:
        feed_dict.update({cp : m for cp, m in zip(gm.conditional_placeholders, batch.metav if isinstance(batch.metav, list) else batch.metav)})
    if gm.weights_placeholder is not None:
        feed_dict[gm.weights_placeholder]        = batch.weights[:, None]
    if gm.pred_norm_real_placeholder is not None:
        feed_dict[gm.pred_norm_real_placeholder] = batch.x_normalization
    return feed_dict

In [204]:
# data0 = gm.sess.run( [gs_assign, gm.gen_output_tensor], feed_dict = feed_dict(training = True)
# data1a  = gm.sess.run( [gs_assign, gm.dis_input_tensor], feed_dict = feed_dict(training = True)
# data1b  = gm.sess.run( [gs_assign, gm.dis_output_real_tensor], feed_dict = feed_dict(training = True)
# data1c  = gm.sess.run( [gs_assign, gm.dis_output_fake_tensor], feed_dict = feed_dict(training = True)

# data2 = gm.sess.run( [gs_assign, gm.mixed_input_data], feed_dict = feed_dict())
# data3 = gm.sess.run( [gs_assign, gm.mixed_output], feed_dict = feed_dict())
# data4 = gm.sess.run( [gs_assign, gm.dis_loss_tensor], feed_dict = feed_dict())
# data5 = gm.sess.run( [gs_assign, gm.dis_calc_grads_op], feed_dict = feed_dict())
data6 = gm.sess.run( [gs_assign, gm.dis_apply_grads_op], feed_dict = feed_dict())
data7 = gm.sess.run( [gs_assign, gm.gen_apply_grads_op], feed_dict = feed_dict())

### Try Foward pass when training isn't on.

In [39]:
with gm.sess.as_default():
    with gm.graph.as_default():
        gen_out_training_false = gm.generator    (gm.gen_input_placeholder, training = False, conditional = gm.conditional_placeholders).data
        dis_out_training_false = gm.discriminator(gen_out_training_false  , training = False, conditional = gm.conditional_placeholders, pred_norm = gm.pred_norm_real_placeholder)

dense_1: units = 200
     x.shape (?, 100, 200)
x_to_samp Tensor("generator/to_sample/transpose_7_3/transpose:0", shape=(?, 1600, 3), dtype=float32)
last_block
<function Discriminator.call at 0x000002826D6CF0D0>
[<tf.Tensor 'discriminator_4/block_7/leaky_re_lu_7/LeakyRelu:0' shape=(?, 100, 1, 32) dtype=float32>] {'training': False, 'conditional': [<tf.Tensor 'conditional:0' shape=(?, 10) dtype=float32>, <tf.Tensor 'conditional_1:0' shape=(?, 7) dtype=float32>], 'pred_norm': <tf.Tensor 'Placeholder:0' shape=(?, 3) dtype=float32>}


In [40]:
data = gm.sess.run(dis_out_training_false, feed_dict = feed_dict(training = False))
data.score.shape

(4, 1)

### Try to run progressive tracking tesors.

In [225]:
with gm.sess.as_default():
    with gm.graph.as_default():
        initial_steps_placeholder, stabilizing_steps_placeholder, stage_int_tensor, fade_phase_bool, alpha_tensor = ProgressiveGAN.get_or_create_tracking_tensors()
        predicate = (stage_int_tensor <= 0) & tf.equal(fade_phase_bool, tf.constant(False))
gm.sess.run([initial_steps_placeholder, stabilizing_steps_placeholder, stage_int_tensor, fade_phase_bool, predicate, alpha_tensor], feed_dict = {global_step : 90000})

[10000, 10000, 5, True, False, 0.0]

### Try to compute grads of mixed output when training is False (i.e. no progressive tensors will be used)

In [114]:
with gm.graph.as_default():
    
    with gm.sess.as_default():

        gen_output = gm.generator(gm.gen_input_placeholder, training = True, conditional = gm.conditional_placeholders)
        gen_output_tensor     = gen_output.data
        pred_norm_fake_tensor = gen_output.pred_norm

        epsilon               = tf.random_uniform([], 0, 1)
        mixed_input_data      = (epsilon * gm.dis_input_placeholder     ) + (1 - epsilon) * gen_output_tensor
        mixed_input_pred_norm = (epsilon * gm.pred_norm_real_placeholder) + (1 - epsilon) * pred_norm_fake_tensor if pred_norm_fake_tensor is not None else None
        mixed_output          = gm.discriminator(mixed_input_data, training = False, conditional = gm.conditional_placeholders, pred_norm = mixed_input_pred_norm).score

        mix_calc_grads_op_data = gm.discriminator_optimizer.compute_gradients(mixed_output, mixed_input_data)

        gm.sess.run(mix_calc_grads_op_data, feed_dict = feed_dict())
    

Generator.call() training = True
     x.shape (?, 100, 3)

dense_1: units = 200
     x.shape (?, 100, 200)
Appending Conditional:
     conditional: Tensor("generator_4/concat:0", shape=(?, 17), dtype=float32) (?, 17)
     conditional: Tensor("generator_4/Tile:0", shape=(?, 17, 3), dtype=float32) (?, 17, 3)
     added_conditional
     x.shape (?, 117, 3)

transpose: perm = [0, 2, 1]
     x.shape (?, 3, 117)
dense_2: units = 200
     x.shape (?, 3, 200)
batch_normalization_1: axis = [1]
     x.shape (?, 3, 200)
transpose_1: perm = [0, 2, 1]
     x.shape (?, 200, 3)
expand_dim: axis = 2
     x.shape (?, 200, 1, 3)
conv2d_1: filters = 3 // kernel_size = (16, 1) // strides = (1, 1) 
     x.shape (?, 200, 1, 3)
batch_normalization_2: axis = [3]
     x.shape (?, 200, 1, 3)
re_lu
     x.shape (?, 200, 1, 3)
conv2d_2: filters = 6 // kernel_size = (16, 1) // strides = (1, 1) 
     x.shape (?, 200, 1, 6)
batch_normalization_3: axis = [3]
     x.shape (?, 200, 1, 6)
re_lu_1
     x.shape (?, 200, 1

InvalidArgumentError: Incompatible shapes: [4,1600,3] vs. [4,100,3]
	 [[node add_9 (defined at <ipython-input-114-159af2f8b2a4>:10) ]]

Caused by op 'add_9', defined at:
  File "C:\Users\micha\AppData\Local\Programs\Python\Python36\Lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\micha\AppData\Local\Programs\Python\Python36\Lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\micha\envs\seis3.6.7\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\users\micha\envs\seis3.6.7\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "c:\users\micha\envs\seis3.6.7\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  File "c:\users\micha\envs\seis3.6.7\lib\site-packages\tornado\platform\asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()
  File "C:\Users\micha\AppData\Loca

### Try to compute gen loss tensors and grad tensors.

In [179]:
batch   = gm.SG_train.random_metav()

feed_dict = {gm.gen_input_placeholder : gm.noise()}
if gm.conditional_placeholders is not None:
    feed_dict.update({cp : m for cp, m in zip(gm.conditional_placeholders, batch.metav if isinstance(batch.metav, list) else batch.metav)})
if gm.weights_placeholder is not None:
    feed_dict[gm.weights_placeholder]     = batch.weights[:, None]

Getting   randomly sampled metav.
Retreived randomly sampled metav.


In [186]:
# gm.sess.run([gm.gen_calc_grads_op], feed_dict = feed_dict)
# gm.sess.run(gm.generator_optimizer.compute_gradients(gm.gen_output_tensor,  gm.generator.variables), feed_dict = feed_dict)

In [187]:
gen_calc_grads_op = gm.generator_optimizer.compute_gradients(gm.gen_output_tensor,  gm.generator.variables)

# Remove null gradients.
gen_grads_idx       = [i for i, g_op in enumerate(gen_calc_grads_op) if g_op[0] is not None]
generator_variables = [gen_calc_grads_op[i][1] for i in gen_grads_idx]
gen_calc_grads_op   = [gen_calc_grads_op[i][0] for i in gen_grads_idx]

# Clip gradients.
if gm.config.get('clip_gen_grad_norm'):
    gen_calc_grads_op = [g_op for g_op in gen_calc_grads_op if g_op[0] is not None] if isinstance(gen_calc_grads_op, list) else gen_calc_grads_op
    gen_calc_grads_op, gen_grads_global_norm = tf.clip_by_global_norm(gen_calc_grads_op, gm.config['clip_gen_grad_norm'])


In [189]:
# gm.sess.run(gen_calc_grads_op, feed_dict = feed_dict) # This works, so grads through the progressive gen are 'A-ok =b'